# Set up the environment

Before running the notebook, please install the necessary packages and environment by running the following shell commands in your terminal:

```bash
# Create the conda environment from the provided environment file
conda env create -f ../conda_env_pkgs.yml -n soc_model_env

# Activate the new environment
conda activate soc_model_env

# Launch Jupyter Notebook from within the environment
jupyter notebook


In [19]:
import os
import json
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score

from pprint import pprint as pp
import tabulate

# Authenticate and Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project= "ee-christopherharrellgis")

# Optional: Display map
#Map = geemap.Map(basemap = "SATELLITE")
Map = geemap.Map()
#Map

In [18]:
# Global variables
output_data_folder = "../soc/data/"

start_year = 2023
end_year = 2024
month_start = 3
month_end = 2
max_cloud_cover = 20

#### Import SOC Samples and Study Area as Feature Collections

In [176]:
# Create a FeatureCollection from SOC_samples.csv
def create_points_feature_collection(df):
    features = []
    for idx, row in df.iterrows():
        point = ee.Geometry.Point([row['longitude'], row['latitude']])
        feature = ee.Feature(point, {
            'plot_no': row['plot_no'],
            'MgC_per_ha': row['MgC_per_ha'],
            'MgC_SE': row['MgC_SE']
        })
        features.append(feature)
    return ee.FeatureCollection(features)

# Load the study area GeoJSON file
study_area_geojson = '../soc/data/study_area.geojson'

# Load the SOC samples table
soc_samples_df = pd.read_csv("../soc/data/SOC_samples.csv")

with open(study_area_geojson) as f:
    geojson_data = json.load(f)

# Convert the study area to an Earth Engine FeatureCollection
study_area = ee.FeatureCollection(geojson_data)

# Convert the SOC samples table to an Earth Engine FeatureCollection
soc_sample_points = create_points_feature_collection(soc_samples_df)

#Map.addLayer(study_area, {'color': 'red'}, 'Study Area', False)
Map.addLayer(soc_sample_points, {'color': 'yellow'}, 'Sample Points', False)
Map.centerObject(study_area, zoom=11)

In [ ]:
""" wdpa = ee.FeatureCollection('WCMC/WDPA/current/polygons')

# Filter WDPA features that spatially intersect with your study area
overlapping_pas = wdpa.filter(ee.Filter.intersects('.geo', soc_sample_points.geometry()))

# Get the list of names
protected_areas_with_names = overlapping_pas.filter(ee.Filter.notNull(['NAME']))

# Get the names as a list
protected_area_names = protected_areas_with_names.aggregate_array('NAME')

# Print the names of protected areas that contain sample points
print('Protected Areas containing sample points:')
print(protected_area_names.getInfo())

Map.addLayer(overlapping_pas, {'color': 'blue'}, 'PAS') """

#### Generate an Image for each environmental covariate
- Elevation (Copernicus DEM)
- Slope (Copernicus DEM)
- Mean Annual Precipitation (MAP) (TerraClimate)
- Mean Annual Temperature (MAT) (TerraClimate)
- Mean NDVI (Sentinel-2 SR)
- Mean EVI (Sentinel-2 SR)
- ESA Landcover Classification (ESA WorldCover)

In [4]:
def print_metadata(rasters: dict):
    # Loop through each raster
    for key, raster in rasters.items():
        # Check the type
        info = raster.getInfo()  # download metadata
        name = key
        type_name = info['type']
        
        print(f'Raster {name}:')
        print(f'\tType: {type_name}')
        
        if type_name == 'Image':
            bands = raster.bandNames().getInfo()
            print(f'\tBands: {bands}')
            for band in bands:
                stats = raster.select(band).reduceRegion(
                    reducer=ee.Reducer.minMax(),
                    geometry=study_area,
                    scale=30,
                    maxPixels=1e13
                )
                print(f'\t{band} min/max:', stats.getInfo())
        
        elif type_name == 'ImageCollection':
            # Get bands from first image in collection
            first_img = raster.first()
            bands = first_img.bandNames().getInfo()
            print(f'\tBands (from first image): {bands}')
            for band in bands:
                stats = raster.select(band).reduceRegion(
                    reducer=ee.Reducer.minMax(),
                    geometry=study_area,
                    scale=30,
                    maxPixels=1e13
                )
                print(f'\t{band} min/max:', stats.getInfo())
        
        else:
            print('Unknown Earth Engine object type.')
        
        print('')


In [9]:

# COPERNICUS DEM (30m)
dem = ee.ImageCollection('COPERNICUS/DEM/GLO30').mosaic().select('DEM')

elevation = dem.reproject(crs='EPSG:4326', scale=10)
slope = ee.Terrain.slope(dem)

# Visualization parameters for Elevation
vis_params_elevation = {
    'min': 0,
    'max': 3000,
    'palette': ['#00FFFF', '#0000FF', '#008000', '#FFFF00', '#FF0000', '#800000']
}

# Visualization parameters for slope
vis_params_slope = {
    'min': 0,
    'max': 60,
    'palette': ['#00FFFF', '#0000FF', '#008000', '#FFFF00', '#FF0000', '#800000']
}

#Map.addLayer(elevation.clip(study_area), vis_params_elevation, "elevation", False)
#Map.addLayer(slope.clip(study_area), vis_params_slope, "slope", False)

topographic_covariates = {
    'elevation': elevation.clip(study_area),
    'slope': slope.clip(study_area)
}

### TerraClimate Temperature and Precipitation

In [10]:
def config_TC_bands(img):
    tmmn_band = img.select('tmmn').multiply(0.1)
    tmmx_band = img.select('tmmx').multiply(0.1)
    precip_band = img.select('pr')
    
    bands = precip_band.addBands([tmmn_band, tmmx_band], overwrite=True)
    return bands.copyProperties(img, img.propertyNames())

def calc_map(img):
    """Computes the mean daily temperature for each month."""
    tmmx = img.select('tmmx')
    tmmn = img.select('tmmn')
    tavg = tmmx.add(tmmn).divide(2).rename('tavg')
    return tavg.copyProperties(img, img.propertyNames())

imgCol_TC = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
    .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
    .filter(ee.Filter.calendarRange(month_start, month_end, 'month')) \
    .filterBounds(study_area) \
    .map(config_TC_bands)

imgCol_TC_temp = imgCol_TC.select(['tmmn', 'tmmx'])
imgCol_TC_precip = imgCol_TC.select(['pr'])

#### Precipitation Aggregations

In [11]:
# Mean monthly precipitation (average monthly precipitation over time period)
tc_monthly_avg_precip = imgCol_TC_precip.select('pr').mean().rename('mean_precip')

# Total accumulated precipitation across the entire time period (sum of all months)
tc_total_precip = imgCol_TC.select(['pr']).sum().rename('total_precip')

# Standard deviation (variability) of monthly precipitation
tc_stddev_precip = imgCol_TC.select('pr').reduce(ee.Reducer.stdDev()).rename('stddev_precip')

# Minimum and maximum monthly precipitation values
tc_min_precip = imgCol_TC.select('pr').min().rename('min_precip')
tc_max_precip = imgCol_TC.select('pr').max().rename('max_precip')

# 10th and 90th percentile monthly precipitation values
tc_p10_precip = imgCol_TC.select('pr').reduce(ee.Reducer.percentile([10])).rename('p10_precip')
tc_p90_precip = imgCol_TC.select('pr').reduce(ee.Reducer.percentile([90])).rename('p90_precip')

# Median monthly precipitation value
tc_median_precip = imgCol_TC.select('pr').median().rename('median_precip')

# Visualization for Precipitation-Related Covariates (pr)
viz_precip = {
    'min': 0,
    'max': 300,
    'palette': ['white', 'blue', 'darkblue']
}

# Visualization for Precipitation Standard Deviation
viz_precip_stddev = {
    'min': 0,
    'max': 100,
    'palette': ['white', 'cyan', 'blue']
}

# Visualization for Total Precipitation
viz_total_precip = {
    'min': 0,
    'max': 2000,
    'palette': ['white', 'green', 'darkgreen']
}

#Add precip covariates
""" Map.addLayer(tc_monthly_avg_precip.clip(study_area), viz_precip, 'tc_monthly_avg_precip', False)
Map.addLayer(tc_total_precip.clip(study_area), viz_total_precip, 'tc_total_precip', False)
Map.addLayer(tc_stddev_precip.clip(study_area), viz_precip_stddev, 'tc_stddev_precip', False)
Map.addLayer(tc_min_precip.clip(study_area), viz_precip, 'tc_min_precip', False)
Map.addLayer(tc_max_precip.clip(study_area), viz_precip, 'tc_max_precip', False)
Map.addLayer(tc_p10_precip.clip(study_area), viz_precip, 'tc_p10_precip', False)
Map.addLayer(tc_p90_precip.clip(study_area), viz_precip, 'tc_p90_precip', False)
Map.addLayer(tc_median_precip.clip(study_area), viz_precip, 'tc_median_precip', False) """

precip_covariates = {
    'tc_monthly_avg_precip': tc_monthly_avg_precip.clip(study_area),
    'tc_total_precip': tc_total_precip.clip(study_area),
    'tc_stddev_precip': tc_stddev_precip.clip(study_area),
    'tc_min_precip': tc_min_precip.clip(study_area),
    'tc_max_precip': tc_max_precip.clip(study_area),
    'tc_p10_precip': tc_p10_precip.clip(study_area),
    'tc_p90_precip': tc_p90_precip.clip(study_area),
    'tc_median_precip': tc_median_precip.clip(study_area),
}


#### Temperature Aggregations

In [12]:
# Mean daily temperature (tavg = (tmmn + tmmx)/2), then averaged across the entire time period
tc_mean_temp = imgCol_TC_temp.map(calc_map).mean()

# Mean minimum and maximum daily temperatures (averaged separately across the entire time period)
tc_mean_tmmn = imgCol_TC.select('tmmn').mean().rename('mean_tmmn')
tc_mean_tmmx = imgCol_TC.select('tmmx').mean().rename('mean_tmmx')

# Standard deviation (variability) of minimum and maximum daily temperatures
tc_stddev_tmmn = imgCol_TC.select('tmmn').reduce(ee.Reducer.stdDev()).rename('stddev_tmmn')
tc_stddev_tmmx = imgCol_TC.select('tmmx').reduce(ee.Reducer.stdDev()).rename('stddev_tmmx')

# Minimum and maximum values of minimum and maximum daily temperatures across the time period
tc_min_tmmn = imgCol_TC.select('tmmn').min().rename('min_tmmn')
tc_max_tmmn = imgCol_TC.select('tmmn').max().rename('max_tmmn')

tc_min_tmmx = imgCol_TC.select('tmmx').min().rename('min_tmmx')
tc_max_tmmx = imgCol_TC.select('tmmx').max().rename('max_tmmx')

# 10th and 90th percentile values of minimum and maximum daily temperatures
tc_p10_tmmn = imgCol_TC.select('tmmn').reduce(ee.Reducer.percentile([10])).rename('p10_tmmn')
tc_p90_tmmn = imgCol_TC.select('tmmn').reduce(ee.Reducer.percentile([90])).rename('p90_tmmn')

tc_p10_tmmx = imgCol_TC.select('tmmx').reduce(ee.Reducer.percentile([10])).rename('p10_tmmx')
tc_p90_tmmx = imgCol_TC.select('tmmx').reduce(ee.Reducer.percentile([90])).rename('p90_tmmx')

# Median (50th percentile) of minimum and maximum daily temperatures
tc_median_tmmn = imgCol_TC.select('tmmn').median().rename('median_tmmn')
tc_median_tmmx = imgCol_TC.select('tmmx').median().rename('median_tmmx')

# Visualization for Temperature-Related Covariates
viz_temp = {
    'min': -10,
    'max': 40,
    'palette': ['blue', 'white', 'red']
}

# Visualization for Temperature Standard Deviation
viz_temp_stddev = {
    'min': 0,
    'max': 10,
    'palette': ['white', 'purple', 'black']
}

#Add temp covariates
""" Map.addLayer(tc_mean_temp.clip(study_area), viz_temp, 'tc_mean_temp', False)
Map.addLayer(tc_mean_tmmn.clip(study_area), viz_temp, 'tc_mean_tmmn', False)
Map.addLayer(tc_mean_tmmx.clip(study_area), viz_temp, 'tc_mean_tmmx', False)
Map.addLayer(tc_stddev_tmmn.clip(study_area), viz_temp_stddev, 'tc_stddev_tmmn', False)
Map.addLayer(tc_stddev_tmmx.clip(study_area), viz_temp_stddev, 'tc_stddev_tmmx', False)
Map.addLayer(tc_min_tmmn.clip(study_area), viz_temp, 'tc_min_tmmn', False)
Map.addLayer(tc_max_tmmn.clip(study_area), viz_temp, 'tc_max_tmmn', False)
Map.addLayer(tc_min_tmmx.clip(study_area), viz_temp, 'tc_min_tmmx', False)
Map.addLayer(tc_max_tmmx.clip(study_area), viz_temp, 'tc_max_tmmx', False)
Map.addLayer(tc_p10_tmmn.clip(study_area), viz_temp, 'tc_p10_tmmn', False)
Map.addLayer(tc_p90_tmmn.clip(study_area), viz_temp, 'tc_p90_tmmn', False)
Map.addLayer(tc_p10_tmmx.clip(study_area), viz_temp, 'tc_p10_tmmx', False)
Map.addLayer(tc_p90_tmmx.clip(study_area), viz_temp, 'tc_p90_tmmx', False)
Map.addLayer(tc_median_tmmn.clip(study_area), viz_temp, 'tc_median_tmmn', False)
Map.addLayer(tc_median_tmmx.clip(study_area), viz_temp, 'tc_median_tmmx', False) """

temp_covariates = {
    'tc_mean_temp': tc_mean_temp.clip(study_area),
    'tc_mean_tmmn': tc_mean_tmmn.clip(study_area),
    'tc_mean_tmmx': tc_mean_tmmx.clip(study_area),
    'tc_stddev_tmmn': tc_stddev_tmmn.clip(study_area),
    'tc_stddev_tmmx': tc_stddev_tmmx.clip(study_area),
    'tc_min_tmmn': tc_min_tmmn.clip(study_area),
    'tc_max_tmmn': tc_max_tmmn.clip(study_area),
    'tc_min_tmmx': tc_min_tmmx.clip(study_area),
    'tc_max_tmmx': tc_max_tmmx.clip(study_area),
    'tc_p10_tmmn': tc_p10_tmmn.clip(study_area),
    'tc_p90_tmmn': tc_p90_tmmn.clip(study_area),
    'tc_p10_tmmx': tc_p10_tmmx.clip(study_area),
    'tc_p90_tmmx': tc_p90_tmmx.clip(study_area),
    'tc_median_tmmn': tc_median_tmmn.clip(study_area),
    'tc_median_tmmx': tc_median_tmmx.clip(study_area),
}

#### Sentinel-2_SR NDVI & EVI

In [13]:
def config_s2_bands(img):
    bands = img.select(['B2', 'B4', 'B8'])
    renamed_bands = bands.rename(['B', 'R', 'NIR'])
    return renamed_bands.copyProperties(img, img.propertyNames())

def mask_s2_clouds(image):
    """Masks clouds and cirrus based on SCL band."""
    scl = image.select(['SCL'])
    cloudShadow = scl.eq(3)
    cloudsLow = scl.eq(7)
    cloudsMed = scl.eq(8)
    cloudsHigh = scl.eq(9)
    cirrus = scl.eq(10)
    mask = (cloudShadow.Or(cloudsLow).Or(cloudsMed).Or(cloudsHigh).Or(cirrus).Not())
    return image.updateMask(mask).divide(10000).copyProperties(image, image.propertyNames())

imgCol_S2_SR = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filter(ee.Filter.calendarRange(start_year, end_year, 'year'))\
    .filter(ee.Filter.calendarRange(month_start, month_end, 'month'))\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .filterBounds(study_area) \
    .map(mask_s2_clouds) \
    .map(config_s2_bands)
    
def calc_ndvi(img):
    ndvi = img.normalizedDifference(['NIR', 'R']).rename('NDVI')
    return ndvi.copyProperties(img, img.propertyNames())

def calc_evi(img):
    nir = img.select('NIR')
    red = img.select('R')
    blue = img.select('B')
    
    numerator = nir.subtract(red)
    denominator = nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)
    
    evi = numerator.divide(denominator).multiply(2.5).rename('EVI')

    return evi.copyProperties(img, img.propertyNames())


mean_ndvi = imgCol_S2_SR.map(calc_ndvi).mean()
mean_evi = imgCol_S2_SR.map(calc_evi).mean()

# EVI visualization
evi_vis = {
    'min': 0.0,
    'max': 1.0,
    'palette': ['purple', 'white', 'green']
}

# NDVI visualization
ndvi_vis = {
    'min': 0.0,
    'max': 1.0,
    'palette': ['blue', 'white', 'green']
}

veg_covariates = {
    'ndvi': mean_ndvi.clip(study_area),
    'evi': mean_evi.clip(study_area)

}

#Map.addLayer(mean_ndvi.clip(study_area), ndvi_vis, 'NDVI', False)
#Map.addLayer(mean_evi.clip(study_area), evi_vis, 'EVI', False)

In [ ]:
landcover = ee.Image('ESA/WorldCover/v200/2021')

# Define the land cover class values
landcover_classes = [
    10,  # Tree cover
    20,  # Shrubland
    30,  # Grassland
    40,  # Cropland
    50,  # Built-up
    60,  # Bare / sparse vegetation
    70,  # Snow and ice
    80,  # Permanent water bodies
    90,  # Herbaceous wetland
    95,  # Mangroves
    100  # Moss and lichen
]

# Define the new 0-based sequential remap values (from 0 to 10)
remap_values = ee.List.sequence(0, 10)

def create_binary_bands(image, class_values, remap_values):
    # Remap the class values to a 0-based sequential series
    remapped_image = image.remap(class_values, remap_values).toByte()
    
    # Initialize an empty list to store the binary bands
    binary_bands = []
    
    # Generate a binary band for each class (0 for absence, 1 for presence)
    for i in range(len(class_values)):
        binary_band = remapped_image.eq(i).rename(f'lc_{i}')  # Check if the class is present (1) or absent (0)
        binary_bands.append(binary_band)
    
    # Combine all binary bands into a multi-band image
    binary_image = ee.Image(binary_bands).rename([f'lc_{i}' for i in range(len(class_values))])
    
    return binary_image


encoded_land_cover = create_binary_bands(landcover, landcover_classes, remap_values)

lc_vis = {
    'bands': ['Map']
}

landcover_covariates = {
    'landcover': landcover.clip(study_area)
}

encoded_landcover_covariates = {
    'landcover': encoded_land_cover
}


In [ ]:
# This script was adapted from an Open-Source Github repo here: https://github.com/leonsnill/lst_landsat/blob/master/lst_landsat.py
# Global variables
#start_year = 2023
#end_year = 2024
#month_start = 3
#month_end = 2
#max_cloud_cover = 20
t_threshold = 0

# Algorithm Specifications
# min/max ndvi
ndvi_v = 0.63
ndvi_s = -0.1

# Veg, soil, water emissivity
epsilon_v = 0.985
epsilon_s = 0.96
epsilon_w = 0.99

# Coefficients for Landsat 8
cs_l8 = [0.04019, 0.02916, 1.01523,
         -0.38333, -1.50294, 0.20324,
         0.00918, 1.36072, -0.27514]

def config_l8_bands(img):
    bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
    thermal_band = ['ST_B10']
    new_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2']
    new_thermal_bands = ['TIR']
    vnirswir = img.select(bands).multiply(0.0001).rename(new_bands)
    tir = img.select(thermal_band).multiply(0.1).rename(new_thermal_bands)
    return vnirswir.addBands(tir).copyProperties(img, ['system:time_start'])

# Cloud mask for Surface Reflectance products
def mask_l8_clouds(img):
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    qa = img.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
           qa.bitwiseAnd(cloudsBitMask).eq(0))
    return img.updateMask(mask)

# Radiometric Calibration
def fun_radcal(img):
    radiance = ee.Algorithms.Landsat.calibratedRadiance(img).rename('RADIANCE')
    return img.addBands(radiance)

# L to ee.Image
def fun_l_addband(img):
    l = ee.Image(img.get('L')).select('RADIANCE').rename('L')
    return img.addBands(l)

# NDVI - required for emissivity calculation
def fun_ndvi(img):
    ndvi = img.normalizedDifference(['NIR', 'R']).rename('NDVI')
    return img.addBands(ndvi)

# FVC (Fraction Vegetation Cover) - required for emissivity calculation
def fun_fvc(img):
    fvc = img.expression(
        '((NDVI-NDVI_s)/(NDVI_v-NDVI_s))**2',
        {
            'NDVI': img.select('NDVI'),
            'NDVI_s': ndvi_s,
            'NDVI_v': ndvi_v
        }
    ).rename('FVC')
    return img.addBands(fvc)

# Scale Emissivity - required for LST calculation
def fun_epsilon_scale(img):
    epsilon_scale = img.expression(
        'epsilon_s+(epsilon_v-epsilon_s)*FVC',
        {
            'FVC': img.select('FVC'),
            'epsilon_s': epsilon_s,
            'epsilon_v': epsilon_v
        }
    ).rename('EPSILON_SCALE')
    return img.addBands(epsilon_scale)

# Emissivity (Epsilon) - required for LST calculation
def fun_epsilon(img):
    pseudo = img.select(['NDVI']).set('system:time_start', img.get('system:time_start'))
    epsilon = pseudo.where(img.expression('NDVI > NDVI_v',
                                         {'NDVI': img.select('NDVI'),
                                          'NDVI_v': ndvi_v}), epsilon_v)
    epsilon = epsilon.where(img.expression('NDVI < NDVI_s && NDVI >= 0',
                                          {'NDVI': img.select('NDVI'),
                                           'NDVI_s': ndvi_s}), epsilon_s)
    epsilon = epsilon.where(img.expression('NDVI < 0',
                                          {'NDVI': img.select('NDVI')}), epsilon_w)
    epsilon = epsilon.where(img.expression('NDVI <= NDVI_v && NDVI >= NDVI_s',
                                          {'NDVI': img.select('NDVI'),
                                           'NDVI_v': ndvi_v,
                                           'NDVI_s': ndvi_s}), img.select('EPSILON_SCALE')).rename('EPSILON')
    return img.addBands(epsilon)

# Scale WV content
def fun_wv_scale(img):
    wv_scaled = ee.Image(img.get('WV')).multiply(0.1).rename('WV_SCALED')
    wv_scaled = wv_scaled.resample('bilinear')
    return img.addBands(wv_scaled)

# Atmospheric Functions - required for LST calculation
def fun_af1(img):
    af1 = img.expression(
        '('+str(cs_l8[0])+'*(WV**2))+('+str(cs_l8[1])+'*WV)+('+str(cs_l8[2])+')',
        {
            'WV': img.select('WV_SCALED')
        }
    ).rename('AF1')
    return img.addBands(af1)

def fun_af2(img):
    af2 = img.expression(
        '('+str(cs_l8[3])+'*(WV**2))+('+str(cs_l8[4])+'*WV)+('+str(cs_l8[5])+')',
        {
            'WV': img.select('WV_SCALED')
        }
    ).rename('AF2')
    return img.addBands(af2)

def fun_af3(img):
    af3 = img.expression(
        '('+str(cs_l8[6])+'*(WV**2))+('+str(cs_l8[7])+'*WV)+('+str(cs_l8[8])+')',
        {
            'WV': img.select('WV_SCALED')
        }
    ).rename('AF3')
    return img.addBands(af3)

# Gamma Function - required for LST calculation
def fun_gamma(img):
    gamma = img.expression('(BT**2)/(1324*L)',
                          {'BT': img.select('TIR'),
                           'L': img.select('L')
                          }).rename('GAMMA')
    return img.addBands(gamma)

# Delta Function - required for LST calculation
def fun_delta(img):
    delta = img.expression('BT-((BT**2)/1324)',
                          {'BT': img.select('TIR')
                          }).rename('DELTA')
    return img.addBands(delta)

# Land Surface Temperature calculation
def fun_lst(img):
    lst = img.expression(
        '(GAMMA*(((1/EPSILON)*(AF1*L+AF2))+AF3)+DELTA)-273.15',
        {
            'GAMMA': img.select('GAMMA'),
            'DELTA': img.select('DELTA'),
            'EPSILON': img.select('EPSILON'),
            'AF1': img.select('AF1'),
            'AF2': img.select('AF2'),
            'AF3': img.select('AF3'),
            'L': img.select('L')
        }
    ).rename('LST')
    return img.addBands(lst)

def fun_mask_lst(img):
    mask = img.select('LST').gt(t_threshold)
    return img.updateMask(mask)

# Create maxDifference-filter to match TOA and SR products
maxDiffFilter = ee.Filter.maxDifference(
    difference=2 * 24 * 60 * 60 * 1000,
    leftField='system:time_start',
    rightField='system:time_start'
)

# Define joins
join_wv = ee.Join.saveBest(
    matchKey='WV',
    measureKey='timeDiff'
)

join_l = ee.Join.saveBest(
    matchKey='L',
    measureKey='timeDiff'
)

# Landsat 8 OLI-TIRS
imgCol_L8_TOA = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
    .filterBounds(study_area)\
    .filter(ee.Filter.calendarRange(start_year, end_year, 'year'))\
    .filter(ee.Filter.calendarRange(month_start, month_end, 'month'))\
    .filter(ee.Filter.lt('CLOUD_COVER_LAND', max_cloud_cover))\
    .select(['B10']) # Thermal Infared 1

imgCol_L8_SR = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    .filterBounds(study_area)\
    .filter(ee.Filter.calendarRange(start_year, end_year, 'year'))\
    .filter(ee.Filter.calendarRange(month_start, month_end, 'month'))\
    .filter(ee.Filter.lt('CLOUD_COVER_LAND', max_cloud_cover))\
    .map(mask_l8_clouds)

imgCol_L8_SR = imgCol_L8_SR.map(config_l8_bands)

# NCEP/NCAR Water Vapor Product
imgCol_WV = ee.ImageCollection('NCEP_RE/surface_wv')\
    .filterBounds(study_area)\
    .filter(ee.Filter.calendarRange(start_year, end_year, 'year'))\
    .filter(ee.Filter.calendarRange(month_start, month_end, 'month'))

# TOA (Radiance) and SR
imgCol_L8_TOA = imgCol_L8_TOA.map(fun_radcal)
imgCol_L8_SR = ee.ImageCollection(join_l.apply(imgCol_L8_SR, imgCol_L8_TOA, maxDiffFilter))
imgCol_L8_SR = imgCol_L8_SR.map(fun_l_addband)

# Water Vapor
imgCol_L8_SR = ee.ImageCollection(join_wv.apply(imgCol_L8_SR, imgCol_WV, maxDiffFilter))
imgCol_L8_SR = imgCol_L8_SR.map(fun_wv_scale)

# Atmospheric Functions
imgCol_L8_SR = imgCol_L8_SR.map(fun_af1)
imgCol_L8_SR = imgCol_L8_SR.map(fun_af2)
imgCol_L8_SR = imgCol_L8_SR.map(fun_af3)

# Delta and Gamma Functions
imgCol_L8_SR = imgCol_L8_SR.map(fun_delta)
imgCol_L8_SR = imgCol_L8_SR.map(fun_gamma)

# Parameters and Indices
imgCol_L8_SR = imgCol_L8_SR.map(fun_ndvi)
imgCol_L8_SR = imgCol_L8_SR.map(fun_fvc)
imgCol_L8_SR = imgCol_L8_SR.map(fun_epsilon_scale)
imgCol_L8_SR = imgCol_L8_SR.map(fun_epsilon)

# LST
imgCol_L8_SR = imgCol_L8_SR.map(fun_lst)
imgCol_L8_SR = imgCol_L8_SR.map(fun_mask_lst)

# Calculate mean LST
mean_lst = imgCol_L8_SR.select(['LST']).mean()

vis_params_lst = {
    'min': -10,
    'max': 30,
    'palette': ['#313695', '#74add1', '#fdae61', '#a50026']
}

#Map.addLayer(mean_lst.clip(study_area), vis_params_lst, 'Mean LST')
lst = {
    'mean_lst': mean_lst
}

print_metadata(lst)

#### Combine covariates into a single multi-band ee.Image

In [ ]:
covariates_dict = {**topographic_covariates, **precip_covariates, **temp_covariates, **veg_covariates, **encoded_landcover_covariates}

def generate_covariate_stack(covariates: dict) -> ee.ImageCollection:
    """Combine a dictionary of ee.Image objects into an ee.ImageCollection."""
    images = [img for img in covariates.values()]
    covariate_stack = ee.ImageCollection(images)
    return covariate_stack

imgCol_covariates = generate_covariate_stack(covariates_dict)
imgStack_covariates = imgCol_covariates.toBands()

#imgCol_covariates
#imgStack_covariates


In [177]:
# Extract pixel values for each point in the FeatureCollection from each band of the image
training_fc = imgStack_covariates.reduceRegions(
    collection=soc_sample_points,
    reducer=ee.Reducer.first(),  # Using 'first' to get the pixel value from each band
    scale=30  # Scale of the image in meters (adjust based on the image's resolution)
)

#### Generate a feature table

In [ ]:
# Extract values at sample point locations
# - `scale` should match the resolution of your covariates (e.g., 30 meters)
""" soc_sample_fields = ['plot_no', 'MgC_per_ha', 'MgC_SE', 'longitude', 'latitude']

sampled = imgStack_covariates.sampleRegions(
    collection=soc_sample_points,
    properties=soc_sample_fields,
    scale=30,
    geometries=False  
)

sampled_dicts = sampled.getInfo()
rows = [feature['properties'] for feature in sampled_dicts['features']]
features_df = pd.DataFrame(rows)
features_df.head() """


In [ ]:
# Optionally export the dataframe to local storage
#features_df.to_csv(os.path.join(output_data_folder, 'features.csv'), index=False)

## Train RF Model in Earth Engine

In [181]:

# Define the predictor variables (covariates) and the target variable (MgC_per_ha)
predictors = [
    '0_DEM', '1_slope', '2_mean_precip', '3_total_precip', '4_stddev_precip',
    '5_min_precip', '6_max_precip', '7_p10_precip', '8_p90_precip', '9_median_precip',
    '10_tavg', '11_mean_tmmn', '12_mean_tmmx', '13_stddev_tmmn', '14_stddev_tmmx',
    '15_min_tmmn', '16_max_tmmn', '17_min_tmmx', '18_max_tmmx', '19_p10_tmmn',
    '20_p90_tmmn', '21_p10_tmmx', '22_p90_tmmx', '23_median_tmmn', '24_median_tmmx',
    '25_NDVI', '26_EVI', '27_lc_0', '27_lc_1', '27_lc_2', '27_lc_3', '27_lc_4',
    '27_lc_5', '27_lc_6', '27_lc_7', '27_lc_8', '27_lc_9', '27_lc_10'
]

target_variable = 'MgC_per_ha'

# Train the classifier (Random Forest)
rf = ee.Classifier.smileRandomForest(numberOfTrees=100, maxNodes=30).setOutputMode('REGRESSION')

trained_rf = rf.train(
    features=training_fc,
    classProperty=target_variable,
    inputProperties=predictors
)

predicted_soc = imgStack_covariates.select(predictors).classify(trained_rf).rename('predicted_SOC')

In [ ]:
vis_soc_prediction = {
    'min': -50,  # Minimum value of predicted SOC (adjust based on your dataset range)
    'max': 50,   # Maximum value of predicted SOC (adjust based on your dataset range)
    'palette': [
        'blue',  # Low values (negative SOC or lower carbon stocks)
        'green', # Mid-range values (average carbon stocks)
        'yellow', # High values (higher carbon stocks)
        'red'     # Highest values (very high carbon stocks)
    ]
}

Map.addLayer(predicted_soc.clip(study_area), vis_soc_prediction, 'SOC_RF_prediction', False)
Map

### Model Validation

In [184]:
# Add a random column to enable splitting
training_fc = training_fc.randomColumn('random')

# Split: 80% for training, 20% for testing
train_split = training_fc.filter(ee.Filter.lt('random', 0.8))
test_split = training_fc.filter(ee.Filter.gte('random', 0.8))

# Re-train the classifier on 80% split
trained_split_rf = rf.train(
    features=train_split,
    classProperty=target_variable,
    inputProperties=predictors
)

# Apply the trained model to the 20% hold-out
test_predicted = test_split.classify(trained_split_rf, 'predicted')

# Compute regression metrics
regression_metrics = test_predicted.errorMatrix(target_variable, 'predicted')

# Apply the model to the test set
test_predicted = test_split.classify(trained_split_rf).map(lambda f: f.set('predicted', f.get('classification')))

# Extract lists of predicted and actual values
observed = test_predicted.aggregate_array(target_variable)
predicted = test_predicted.aggregate_array('predicted')

# Compute metrics using ee.Reducer
joined = ee.Array.cat([observed, predicted], axis=1)

# Mean of observed
mean_obs = ee.Number(observed.reduce(ee.Reducer.mean()))

# Compute Total Sum of Squares (SST)
sst = observed.map(lambda val: ee.Number(val).subtract(mean_obs).pow(2))
sst_sum = ee.List(sst).reduce(ee.Reducer.sum())

# Compute Residual Sum of Squares (SSR)
residuals = observed.zip(predicted).map(
    lambda pair: ee.Number(ee.List(pair).get(0)).subtract(ee.Number(ee.List(pair).get(1))).pow(2)
)
ssr_sum = ee.List(residuals).reduce(ee.Reducer.sum())

# R² = 1 - SSR / SST
r2 = ee.Number(1).subtract(ee.Number(ssr_sum).divide(sst_sum))

# RMSE = sqrt(mean((obs - pred)^2))
rmse = ee.Number(ssr_sum).divide(observed.length()).sqrt()

# Print results
print('R²:', r2.getInfo())
print('RMSE:', rmse.getInfo())

R²: 0.302576377380267
RMSE: 22.150140617060593


### Train a model with sklearn

In [ ]:
# Define features (X) and target variable (y)
X = covariate_df[['elevation', 'evi', 'landcover', 'ndvi', 'pr', 'slope', 
        'std_pr', 'std_tmmn', 'std_tmmx']]  # Covariates
y = covariate_df['MgC_per_ha']  # Target variable (MgC_per_ha)

# Optional: Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Get feature importances
feature_importances = rf.feature_importances_

# Create a DataFrame for feature importances
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort the features by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)


| Feature   |   Importance |
|:----------|-------------:|
| std_tmmn  |     0.230555 |
| elevation |     0.165769 |
| std_pr    |     0.160722 |
| evi       |     0.131279 |
| ndvi      |     0.121691 |

R^2 score on test set: 0.23630450247031576

In [ ]:
# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.xlabel('Feature Importance')
plt.title('Feature Importances for Predicting MgC_per_ha')
plt.show()

In [ ]:
# Define the top 5 features based on importance
top_features = ['elevation', 'std_tmmn', 'std_pr', 'evi', 'ndvi']

# Create the selected_covariates image stack
selected_covariates = covariates.select(top_features)

# Sample the covariates and SOC values from your soil samples
X = covariate_df[['elevation', 'std_tmmn', 'std_pr', 'evi', 'ndvi']].values
y = covariate_df['MgC_per_ha'].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Check the R^2 score on the test set
test_score = rf_model.score(X_test, y_test)
print(f"Test R^2 score: {test_score}")